## Initialization

In [1]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
GPU run command:
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10_cnn.py
It gets down to 0.65 test logloss in 25 epochs, and down to 0.55 after 50 epochs.
(it's still underfitting at that point, though).
Note: the data was pickled with Python 2, and some encoding issues might prevent you
from loading it in Python 3. You might have to load it in Python 2,
save it in a different format, load it in Python 3 and repickle it.
'''

from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils

batch_size = 32
nb_classes = 10
nb_epoch = 1
data_augmentation = True
img_shape = (32, 32, 3)

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 4007)


## Model

In [2]:

model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=img_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# let's train the model using SGD + momentum (how original).
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
sgd = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

## Data

In [3]:
import pickle
import gzip

X_train, Y_train, X_test, Y_test = pickle.load(gzip.open("data.pkl.gz", 'rb')) # use normal pickle / cPickle for unpacking

In [4]:
# this will do preprocessing and realtime data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train) # cheating this time. usually, certain amount of random sampling would be better

## Training

In [5]:
%load_ext snakeviz

In [6]:
%%snakeviz

## Generator pattern
model.fit_generator(datagen.flow(X_train, Y_train,
                    batch_size=batch_size),
                    samples_per_epoch=X_train.shape[0],
                    nb_epoch=nb_epoch,
                    validation_data=(X_test, Y_test))

Epoch 1/1
50000/50000 [==============================] - 58s - loss: 1.8402 - acc: 0.3140 - val_loss: 1.4575 - val_acc: 0.4687
 
*** Profile stats marshalled to file '/tmp/tmpgh8jvm2x'. 


In [7]:
%%snakeviz

## Train-on-batch pattern
from tqdm import tqdm
flow = datagen.flow(X_train, Y_train, batch_size=batch_size)
num_batch_epoch = len(X_train) // batch_size
for epoch in range(nb_epoch):
    for it in tqdm(range(num_batch_epoch)):
        x, y = flow.next()
        model.train_on_batch(x, y)
    print(model.evaluate(X_test, Y_test, batch_size=batch_size))

100%|██████████| 1562/1562 [01:20<00:00, 19.47it/s]

  544/10000 [>.............................] - ETA: 1s

 9952/10000 [============================>.] - ETA: 0s[1.3369499954223634, 0.51990000000000003]
 
*** Profile stats marshalled to file '/tmp/tmpjtyh7w1t'. 


In [8]:
model.evaluate(X_test, Y_test)

 9952/10000 [============================>.] - ETA: 0s

[1.3369499954223634, 0.51990000000000003]

## Exercise: Visualize the result

1) Access to the server via SSH

better to use local forward like;
    
    Host ilect.aws
    HOSTNAME xx.xx.xx.xx
    User ubuntu
    IdentityFile ~/.ssh/sshkey.txt
    LocalForward    8888    localhost:8888
 
2) execute following command

    $ snakevis -s -p 8888 -H 0.0.0.0 /tmp/xxx
    (-s for server mode)
    
3) access via browser

    http://xx.xx.xx.xx:8888/...
    (or localhost:8888/...)
    